In [19]:
%matplotlib auto
# %matplotlib auto
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2

import sys
import os.path
print(os.getcwd())
sys.path.insert(0, os.getcwd()+"/../../../../libs/tacx")
print(os.getcwd()+"/../../../../libs/tacx")
from SEMContour import *
from MxpStage import MxpStageXmlParser
sys.path.insert(0, os.getcwd()+"/../../../../libs/imutil")
from ImGUI import imread_gray
sys.path.insert(0, os.getcwd()+"/../../../../libs/common")
from PlotConfig import *
from FileUtil import gpfs2WinPath

CWD = r'C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1'
CWD = gpfs2WinPath(CWD)

inxml = os.path.join(CWD, r'contourselcal430out.xml')

def loadPatternData(imgfile='', contourfile=''):
    bSucceedReadCt, bSucceedReadIm = False, False

    # read contour
    contour = SEMContour()
    bSucceedReadCt = contour.parseFile(contourfile)
    if not bSucceedReadCt:
        raise OSError("Error, contourfile('{}') cannot be parsed".format(contourfile))
        
    im = None
    try:  # read image
        im, _ = imread_gray(imgfile)
        im = (im/65535).astype(np.float32)
        im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
    except:
        # raise
        pass
    return im, contour

def getContourClassifierData(inxml):
    ocf_parser = MxpStageXmlParser(inxml) #, 'outxml'
    # icf = icf_parser.icf
    df_ocf = ocf_parser.occfs2df()
    patternid = 461
    df_occf = df_ocf.loc[df_ocf.name==patternid, :]
    sr_occf = pd.Series(df_occf.values.flatten(), index=df_occf.columns)
    contourfile = sr_occf.loc['contour/path']
    contourfile = CWD + '/' + contourfile
    imgfile = sr_occf.loc['image/path']
    imgfile = CWD + '/' + imgfile
    im, contour = loadPatternData(imgfile, contourfile)
    return im, contour
    # TODO, add bbox plot
    # df.filter(regex='bbox/outlier', axis=1)
im, contour = getContourClassifierData(inxml)

MxpStage DEBUG   : ocf tag: result
MxpStage DEBUG   : No summary tag in input xml： C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1\contourselcal430out.xml


Using matplotlib backend: Qt5Agg
C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\unittest
C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\unittest/../../../../libs/tacx
Warning, can't find 'gpfs' in input file: C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1
bbox in origin image: [189, 189, 835, 835]


In [64]:

def plotContourClassifier(im, contour, wndname=''):
    # plot image and classified contour point
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    imw, imh = contour.getshape()
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    ax.set_title(wndname)

    df = contour.toDf()
    UserGood = df.UserLabel == 1
    FP = (df.UserLabel==1) & (df.clf_label==0)
    FN = (df.UserLabel==0) & (df.clf_label==1)
    ax.imshow(im)
    ax.plot(df.loc[UserGood ,'offsetx'], df.loc[UserGood, 'offsety'], 'b.', markersize=1, label='User Label as good')
    ax.plot(df.loc[~UserGood ,'offsetx'], df.loc[~UserGood, 'offsety'], 'r.', markersize=2, label='User Label as bad'),
    ax.plot(df.loc[FP ,'offsetx'], df.loc[FP, 'offsety'], alpha=0.5,
            linestyle='None', marker= 'o', markersize=4, markeredgewidth=1, markerfacecolor='none', 
            label='FP, UserLable=1 & ClfLabel=0: {}'.format(len(df.loc[FP,:])))
    ax.plot(df.loc[FN ,'offsetx'], df.loc[FN, 'offsety'], alpha=0.5,
            linestyle='None', marker= 'o', markersize=4, markeredgewidth=1, markerfacecolor='none', 
            label='FN, UserLable=0 & ClfLabel=1: {}'.format(len(df.loc[FN,:])))
    #ax = plt.gca() # gca() function returns the current Axes instance
    #ax.set_ylim(ax.get_ylim()[::-1]) # reverse Y
    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()

In [65]:
plotContourClassifier(im, contour, 'Pattern 461')

In [66]:
def plotAllContourClfData(inxml):
    ocf_parser = MxpStageXmlParser(inxml) #, 'outxml'
    # icf = icf_parser.icf
    df_ocf = ocf_parser.occfs2df()
    for _, row in df_ocf.iterrows():
        patternid = row.loc['name']
        usage = row.loc['usage']
        print("{} {}".format(patternid, usage))
        
        contourfile = row.loc['contour/path']
        contourfile = CWD + '/' + contourfile
        imgfile = row.loc['image/path']
        imgfile = CWD + '/' + imgfile
        im, contour = loadPatternData(imgfile, contourfile)
        plotContourClassifier(im, contour, "Pattern "+str(patternid))

In [67]:
plotAllContourClfData(inxml)

MxpStage DEBUG   : ocf tag: result
MxpStage DEBUG   : No summary tag in input xml： C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1\contourselcal430out.xml


1 training
bbox in origin image: [110, 110, 914, 914]
2 training
bbox in origin image: [149, 149, 875, 875]
444 training
bbox in origin image: [166, 166, 858, 858]
461 validation
bbox in origin image: [189, 189, 835, 835]
1001 validation
bbox in origin image: [132, 132, 892, 892]
3658 test
bbox in origin image: [129, 129, 895, 895]
